<a target="_blank" href="https://colab.research.google.com/github/alejandrogtz/cccs630-fall2023/blob/main/module10/markov_models.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Markov Models

## Introduction

In this module, we will explore a different type of model, a Markov model. Markov models are useful for studying the behaviour or decisions of actors or agents in a complex system. 

In preparation for the interaction portion, please watch the following video published by Harvard Online.

In [1]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/JHwyHIz6a8A" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

## Concepts

You will find a list of important concepts we will review in the module below.

- Markov chain
- Markov model
- Stochastic model
- Transition matrix

## Interaction

Markov models are mathematical representations of stochastic systems or processes that evolve over time. A stochastic system is a random system we can describe and study using probabilistic methods.

There are multiple types of Markov models. In this interaction, we will review the most fundamental type: Markov chains. A Markov chain consists of a sequence of states, transitions between these states, and a probability that the transition occurs. As a result, the connected states form a chain or network. The connections between states and transition probabilities are described using a transition matrix.

One of the main characteristics of Markov models is that the future state depends only on its current state and not on its previous states. This is known as the Markov property. Transitions between states are conditioned or dependent only on the state the system is in before the transition occurs.

For example, let's model a system with four states using a Markov chain: S<sub>1</sub>, S<sub>2</sub>, S<sub>3</sub>, and S<sub>4</sub>. The transition probabilties of this four states system can be represented using the following matrix.

$$
P = \begin{pmatrix}
p_{11} & p_{12} & p_{13} & p_{14} \\
p_{21} & p_{22} & p_{23} & p_{24} \\
p_{31} & p_{23} & p_{33} & p_{34} \\
p_{41} & p_{24} & p_{43} & p_{44}
\end{pmatrix}
$$

Then, the probabilities are replaced with actual numerical values obtained from the system, as in the following matrix. The sum of the probabilities of each row should total 1, which represents 100%.

$$
P = \begin{pmatrix}
0.2 & 0.3 & 0.4 & 0.1 \\
0.3 & 0.2 & 0.1 & 0.4 \\
0.4 & 0.1 & 0.3 & 0.2 \\
0.1 & 0.4 & 0.2 & 0.3 \\
\end{pmatrix}
$$

Now, let's assume the system is in stage S<sub>2</sub>. This means we must use the probabilities of the second row in the matrix. If we want to predict the next stage of the system, there is a 30% probability the next stage is S<sub>1</sub>, 20% the system will stay in S<sub>2</sub>, 10% of transitioning to S<sub>3</sub>, and 40% of transitioning to S<sub>4</sub> next.

### Instructions

- Create a Markov chain to study the web browsing behaviour of a user.
- Use the categories assigned to the visited domains to create the model.
- Select an initial state and predict the following states.
- Identify relevant behaviour patterns. 

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation

In [ ]:
rc('animation', html='html5') # Specify the type of animation to be rendered

In [ ]:
data = pd.read_excel('module09_data.xlsx', sheet_name='data') # Load the data into Jupyter

In [ ]:
panelist_ids = data.groupby('panelist_id').size().reset_index(name='counts') # Group users by ID

In [ ]:
panelist_ids

In [ ]:
"""
Adjust the panelist_id value to select the user you want to analyze
"""
user_data = data[data['panelist_id'] == 1137] # Filter a single user 

In [ ]:
user_data

In [ ]:
categories = user_data.groupby('category1').size().reset_index(name='counts') # Group records by category1

In [ ]:
"""
Enter the number of categories, states in the Markov model, you want to use
"""
markov_states = 5

In [ ]:
# Select the top categories based on the entered number of Markov states
categories = categories.sort_values(by=['counts'], ascending=False).head(markov_states)

In [ ]:
categories

In [ ]:
nodes = categories['category1'].tolist()

In [ ]:
nodes

In [ ]:
# Convert the sequential data into a data structure that can be used to create a network model
connections = {
    'start_node': [],
    'end_node': []
}

for index, row in user_data.iterrows():
    
    record = user_data.loc[user_data['prev_id'] == row['id']]
    
    if (len(record)>0):
        if (row['category1'] in nodes) & (record.iloc[0]['category1'] in nodes):
            connections['start_node'].append(row['category1'])
            connections['end_node'].append(record.iloc[0]['category1'])

In [ ]:
connections

In [ ]:
connections = pd.DataFrame.from_dict(connections) # Create a dataframe from a dictionary

In [ ]:
connections

In [ ]:
connections = connections.groupby(['start_node','end_node']).size().reset_index(name='count')

In [ ]:
connections

In [ ]:
# Create a directional graph
G = nx.DiGraph()

# Add nodes and connections to the graph
for index, row in connections.iterrows():
    if (not G.has_node(row['start_node'])):
        G.add_node(row['start_node'])
    if (not G.has_node(row['end_node'])):
        G.add_node(row['end_node'])
    G.add_edge(row['start_node'],row['end_node'], weight=row['count'])

In [ ]:
print('Nodes: ',G.number_of_nodes())

In [ ]:
print('Edges: ',G.number_of_edges())

In [ ]:
pos = nx.spiral_layout(G)

In [ ]:
fig = plt.figure(1, figsize=(10, 10), dpi=50)
nx.draw(G, pos, with_labels=True, node_color='lightblue', font_weight='normal', node_size=1500, width=1)

In [ ]:
nx.degree_centrality(G)

In [ ]:
nx.betweenness_centrality(G)

In [ ]:
nx.closeness_centrality(G)

Creation and estimation of the transition matrix.

In [ ]:
transition_matrix = np.zeros((len(nodes), len(nodes))) 

In [ ]:
transition_matrix

In [ ]:
# Estimation of the transition probabilities
for i in range(len(nodes)):

    total = connections[connections['start_node'] == nodes[i]]['count'].sum()
    for x in range(len(nodes)):
        
        w = connections[(connections['start_node'] == nodes[i]) & (connections['end_node'] == nodes[x])]['count'].head(1)
        value = w.tolist()
        if (len(value)>0):
            transition_matrix[i,x] = value[0]/total

In [ ]:
transition_matrix

Prediction of the next states based on the current state.

In [ ]:
states = nodes

In [ ]:
class MarkovChain:
    def __init__(self, transition_matrix, states):
        self.transition_matrix = np.array(transition_matrix)
        self.states = states
        self.index_dict = {self.states[index]: index for index in range(len(self.states))}
        self.state_dict = {index: self.states[index] for index in range(len(self.states))}

    def next_state(self, current_state):
        return np.random.choice(
            self.states, 
            p=self.transition_matrix[self.index_dict[current_state], :]
        )

    def generate_states(self, current_state, no=10):
        future_states = []
        for i in range(no):
            next_st = self.next_state(current_state)
            future_states.append(next_st)
            current_state = next_st
        return future_states

In [ ]:
markov_chain = MarkovChain(transition_matrix=transition_matrix, states=states)

In [ ]:
"""
Enter the initial state of the Makov chain
"""
initial_state = 'business'

In [ ]:
print("Next state after", initial_state, ":", markov_chain.next_state(current_state=initial_state))

In [ ]:
"""
Enter the number of steps (or decisions) you want to simulate
"""
steps = 100

In [ ]:
simulation_results = markov_chain.generate_states(current_state=initial_state, no=steps)

In [ ]:
print("Next,", steps, "states starting from", initial_state ,":", simulation_results)

How likely the user will go to a determined state assume we start in state X
Assuming a 3 changes of states, how likely we will see a certain combination.


In [ ]:
distributions = np.zeros((steps, len(states))) 

In [ ]:
cont = 0
for x in simulation_results:
    if (cont>0):
        distributions[cont][:] = distributions[cont-1][:]
    index = nodes.index(x)
    distributions[cont][index] = distributions[cont][index]+1
    cont = cont+1

In [ ]:
distributions[steps-1]

In [ ]:
y_limit = max(distributions[steps-1])

In [ ]:
plt.rcParams['savefig.facecolor'] = 'white' # Eliminates the distortion of the graph text

fig, ax = plt.subplots(figsize=(10,9))

N = len(states) # Number of bars

data = np.zeros(len(states)) # Initial data

bars = ax.bar(states, data) # Setting up the bar chart

ax.tick_params(axis='x', labelrotation=30)
# Eventually, this shouldn't be needed and an `ha` argument should
# be available for the above.
plt.xticks(ha='right')

ax.set_ylim(0, y_limit) # Set the axis limits

step = [0] # Keep track of the frame (chart) we want to visualize 

def update(frame, step):

    data = distributions[step[0]][:] 

    for bar, h in zip(bars, data):
        bar.set_height(h)
        
    step[0] += 1  # Increase the step value by 1
    return bars

# TODO ADJUST A -2 inicial -1

ani = FuncAnimation(fig, update, frames=steps-2, interval=100, fargs=(step,), repeat=False) # Create the animation

In [ ]:
# Display the animation.
ani

## Assignment 

### Conceptual Option

Pending.

### Hands-on Option

Pending.

Adjust the number of states or nodes.

## Optional Readings

You will find additional resources in case you would like to continue exploring the topics covered in this module below.

Pending.